<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/087_AgentZero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ✅ Agent System Components (with Planning & Reflection)

1. **Tool** – Function that does something useful (e.g., `create_plan`, `track_progress`, domain tools)
2. **Tool Registry** – Stores all tools by name for lookup by the agent
3. **ActionContext** – Holds dependencies (e.g., memory, auth, services, registry)
4. **Environment** – Executes tools using the ActionContext for dependency injection
5. **Agent** – Parses input, chooses tools, calls them via Environment
6. **Capabilities** – Modular behaviors that hook into the agent loop (e.g., **PlanFirstCapability**, **ProgressTrackingCapability**)
7. **Dependencies** – Injected values (e.g., SMTP, tokens, LLM) automatically provided to tools
8. **Wiring** – Instantiate and connect everything together (Agent + Environment + Capabilities + Tools)

# 4) Environment


## What the Environment does (in plain terms)

* **Looks up the tool** by name in your registry (router).
* **Passes in the ActionContext** automatically (so tools don’t fetch globals).
* **Forwards the args** exactly as the model/tool-call suggests.
* **Hands back the tool’s result** unchanged (for now).

That’s it. It’s just the **switchboard** between “the agent decided to use tool X” and “run tool X with these inputs.”

## Why it matters

* Keeps your **agent loop** tiny: “decide → call env → get result”.
* Central place to add **validation, error handling, logging, DI tricks** later—without touching tools.

## What to look for in the code

* A **single method** like `call(tool_name, args)` that:

  1. pulls the function from the registry,
  2. calls it with `ActionContext` first,
  3. returns whatever the tool returns.
* No business logic here. No memory edits here. Just routing.

## Tiny checklist (use as you read)

* Does it **only**: (1) lookup, (2) call, (3) return?
* Does every tool get **ActionContext** as the first argument?
* Are args passed as `**(args or {})` (so empty dict works)?
* Is this the **only place** tools are executed?

## Optional (next, when ready—still small)

* Wrap calls with `try/except` and normalize results:

  * On success → return `{"ok": True, **tool_result}`
  * On error → return `{"ok": False, "error": str(e)}`
* Add simple **input checks** (later) using the schemas from your decorator.



In [ ]:
class Environment:
    """
    Executes tools with the ActionContext.
    Later: add JSON Schema validation + try/except.
    """
    def __init__(self, registry: dict, action_context: ActionContext):
        self.registry = registry
        self.ctx = action_context

    def call(self, tool_name: str, args: dict) -> dict:
        fn = self.registry[tool_name]       # or: get_tool(tool_name)
        return fn(self.ctx, **(args or {})) # tools always get ActionContext first


# 5) Agent

## What the Agent is responsible for (and only that)

* **Owns `state`** for the run (e.g., `state["current_plan"]`).
* **Asks a policy** (`decide_fn`) what to do next → `(tool_name, args)`.
* **Runs hooks** so capabilities can steer (`before_tool`) and react (`after_tool`).
* **Calls tools via Environment** (so the Agent never touches the registry or deps directly).
* **Updates state** from tool outputs (minimal and explicit).
* **Stops** on a simple, readable rule.

## The loop—mentally step through it

1. **Policy** says: “next do `X` with `{args}`”.
2. **before\_tool hook** may redirect/modify (e.g., force `create_plan`).
3. **Environment** executes the tool with `ActionContext`.
4. **Agent** does small state updates (e.g., write `plan` into `state`).
5. **after\_tool hook** logs/guards/etc. (e.g., progress mirroring).
6. **Stop** when a simple condition is met (e.g., after `summarize_progress`).

If you can narrate those six steps out loud, you’ve got the Agent.

## Separation of concerns (keep these lines bright)

* **Agent**: orchestration + tiny state writes.
* **Environment**: executes tools, passes `ActionContext`.
* **Tools**: do work; read/write `ActionContext.memory`.
* **Capabilities**: policies via **hooks** (no business logic).

## What to check in your code

* **Hooks in the right places?** `before_tool` → *before* env call, `after_tool` → *after*.
* **State updates minimal?** Only copy what the tool returned (e.g., the plan).
* **No hidden deps?** Agent doesn’t import tools or globals—only talks to `env`.
* **Stop rule clear?** A single, obvious condition (you can refine later).

## Micro-exercises (1–2 min each)

* Change the stop rule to “stop after first progress entry exists.” Does the loop still read clearly?
* Temporarily remove `PlanFirstCapability`. What happens on the first iteration?
* Pretend `update_progress` fails (`{"ok": False, "error": "…"}).` Where would you surface that later—Agent or Capability?

## What *not* to do here (on purpose)

* Don’t validate inputs here → that’s a future **Environment** or schema job.
* Don’t cram business logic here → that lives in **Tools**.
* Don’t mutate `ActionContext` from the Agent → that’s for **Tools** and **Environment**.




In [ ]:
class Agent:
    """
    Tiny agent:
    - holds a small `state` dict for the run
    - asks a decide_fn what to do next (tool_name, args)
    - lets capabilities intercept via hooks
    - calls tools through Environment
    - updates state from tool outputs (minimal)
    """
    def __init__(self, env, capabilities: list):
        self.env = env                  # your Environment
        self.capabilities = capabilities  # e.g., [PlanFirstCapability(), ProgressTrackingCapability()]

    def run(self, goal: str, decide_fn, max_iters: int = 4):
        state = {"goal": goal}

        for _ in range(max_iters):
            # 1) model policy (you provide this; returns (tool_name, args))
            tool_name, args = decide_fn(state)

            # 2) HOOK: before_tool (capabilities can redirect/modify)
            for cap in self.capabilities:
                tool_name, args = cap.before_tool(state, tool_name, args)

            # 3) execute tool via Environment (ActionContext is handled by env)
            result = self.env.call(tool_name, args)

            # 4) minimal state updates owned by the agent
            if tool_name == "create_plan" and isinstance(result, dict) and "plan" in result:
                state["current_plan"] = result["plan"]

            # 5) HOOK: after_tool (capabilities can log/guard/trigger)
            for cap in self.capabilities:
                cap.after_tool(state, tool_name, result)

            # 6) super-simple stop rule (adjust later as you learn)
            if tool_name == "summarize_progress":
                break

        return state


In [ ]:
#===================
# Tool Registry
#===================

TOOL_REGISTRY: dict[str, callable] = {}

def register_tool(name: str | None = None):
    """Decorator: auto-register a function as a tool."""
    def deco(func):
        TOOL_REGISTRY[name or func.__name__] = func
        return func
    return deco

# (optional) helpers
def list_tools() -> list[str]: return sorted(TOOL_REGISTRY.keys())
def get_tool(name: str): return TOOL_REGISTRY[name]

#==================
# ActionContext
#==================

from dataclasses import dataclass, field
from typing import Any, Dict, Optional

@dataclass
class ActionContext:
    """
    Per-run backpack for tools.
    - memory: short-term data tools share this run (e.g., goals, progress)
    - deps: injected services/utilities (e.g., clock, clients, tokens)
    - config: knobs/flags for this run (e.g., max_iterations)
    """
    memory: Dict[str, Any] = field(default_factory=dict)
    deps: Dict[str, Any] = field(default_factory=dict)
    config: Dict[str, Any] = field(default_factory=dict)

    # (optional) tiny helpers
    def get_dep(self, name: str, default: Optional[Any] = None) -> Any:
        return self.deps.get(name, default)

    def remember(self, key: str, value: Any) -> None:
        self.memory[key] = value

#==========
# Tools
#==========

@register_tool()  # name defaults to "set_goals"
def set_goals(action_context: ActionContext, goals: list[str]) -> dict:
    # would write: action_context.memory["goals"] = list(goals)
    return {"goals": goals}

@register_tool()  # defaults to "create_plan"
def create_plan(action_context: ActionContext) -> dict:
    # would read: action_context.memory["goals"]
    # agent loop will store into: state["current_plan"]
    return {"plan": ["Step 1 ...", "Step 2 ...", "Step 3 ..."]}

@register_tool()  # defaults to "update_progress"
def update_progress(action_context: ActionContext,
                    step_index: int,
                    status: str,
                    note: str = "") -> dict:
    # would append to: action_context.memory["progress"]
    return {"ok": True, "entry": {"step": step_index, "status": status, "note": note}}


@register_tool(name="summarize_progress")  # explicit name (optional)
def summarize_progress(action_context: ActionContext) -> dict:
    # would read: state["current_plan"] + action_context.memory["progress"]
    return {"summary": "1 in progress, 2 remaining..."}


#==============
# Capability
#==============

class PlanFirstCapability:
    # HOOK: runs before any tool executes
    def before_tool(self, state, tool, args):
        if "current_plan" not in state and tool != "create_plan":
            # redirect to create_plan with no args (matches tool signature)
            return "create_plan", {}
        return tool, args

    def after_tool(self, state, tool, result):
        pass  # no-op

class ProgressTrackingCapability:
    def before_tool(self, state, tool, args):
        return tool, args  # no-op

    # HOOK: runs after a tool completes
    def after_tool(self, state, tool, result):
        if tool == "update_progress" and result.get("ok", True):
            entry = result.get("entry")
            if entry:
                state.setdefault("progress", []).append(entry)

#==============
# Environment
#==============

class Environment:
    """
    Executes tools with the ActionContext.
    Later: add JSON Schema validation + try/except.
    """
    def __init__(self, registry: dict, action_context: ActionContext):
        self.registry = registry
        self.ctx = action_context

    def call(self, tool_name: str, args: dict) -> dict:
        fn = self.registry[tool_name]       # or: get_tool(tool_name)
        return fn(self.ctx, **(args or {})) # tools always get ActionContext first

#===========
# Agent
#===========

class Agent:
    """
    Tiny agent:
    - holds a small `state` dict for the run
    - asks a decide_fn what to do next (tool_name, args)
    - lets capabilities intercept via hooks
    - calls tools through Environment
    - updates state from tool outputs (minimal)
    """
    def __init__(self, env, capabilities: list):
        self.env = env                  # your Environment
        self.capabilities = capabilities  # e.g., [PlanFirstCapability(), ProgressTrackingCapability()]

    def run(self, goal: str, decide_fn, max_iters: int = 4):
        state = {"goal": goal}

        for _ in range(max_iters):
            # 1) model policy (you provide this; returns (tool_name, args))
            tool_name, args = decide_fn(state)

            # 2) HOOK: before_tool (capabilities can redirect/modify)
            for cap in self.capabilities:
                tool_name, args = cap.before_tool(state, tool_name, args)

            # 3) execute tool via Environment (ActionContext is handled by env)
            result = self.env.call(tool_name, args)

            # 4) minimal state updates owned by the agent
            if tool_name == "create_plan" and isinstance(result, dict) and "plan" in result:
                state["current_plan"] = result["plan"]

            # 5) HOOK: after_tool (capabilities can log/guard/trigger)
            for cap in self.capabilities:
                cap.after_tool(state, tool_name, result)

            # 6) super-simple stop rule (adjust later as you learn)
            if tool_name == "summarize_progress":
                break

        return state
